# SVMジェスチャー推論

- 同じディレクトリに`microbit_data2.csv`が必要
- 同じディレクトリに`model.sav`も必要

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd

#pandasでcsv読込み
df = pd.read_csv('microbit_data2.csv', header=None)

#カラム名を付与
df.columns = ['x', 'y', 'z', 'label']

#プロット
df.plot(y=df.columns[0:3])

In [ ]:
#移動平均と標準偏差のデータに変換
import numpy as np

#移動平均と標準偏差で計算対象とするデータ値
n = 50

#x,y,zのデータ列のみ抜粋
df_xyz = df.iloc[:, 0:3]

#移動平均
df_mean = df_xyz.rolling(n).mean() #n個ずつのデータで平均
df_mean.columns = ['mean_x', 'mean_y', 'mean_z'] #列名を変更

#標準偏差
df_std = df_xyz.rolling(n).std() #n個ずつのデータで標準偏差
df_std.columns = ['std_x', 'std_y', 'std_z'] #列名を変更

#移動平均と標準偏差を横に結合
df_xyz = df_mean.join(df_std)

#NaN（欠損値）データが１つでも含まれる行は削除
df_xyz = df_xyz.dropna()

#プロット
df_xyz.plot(y=df_xyz.columns[0:3], title='test')
df_xyz.plot(y=df_xyz.columns[3:6], title='test')

#データをndarrayに変換してX_testとする
X_test = df_xyz.values

#確認表示
print(X_test)

In [ ]:
from sklearn.svm import LinearSVC
import pickle

#モデル読み込み
model = pickle.load(open('model.sav', 'rb'))

#推論
y_pred = model.predict(X_test)

#結果表示
print(y_pred)

In [ ]:
from scipy.special import softmax

#各ラベルに対するスコアを出す
y_decition = model.decision_function(X_test)
print(y_decition)

#softmaxを使って確率値に近いものに変換
np.set_printoptions(precision=2, suppress=True)#小数点以下2桁表示で指数表記を禁止
y_prob = softmax(y_decition, axis=1)
print(y_prob)

#DataFrameに変換
df_y_prob = pd.DataFrame(y_prob)
print(df_y_prob)

In [ ]:
#列名をラベルに紐づいたテキストにする
gestures = ['Standing (stationary)', 'Walking','Sit', 'Keystroke', 'Bye bye', ' Arms folded', 'Guts pose', 'Scratch the back of the head', 'Thumbs up']
label_texts = []

#学習時に使ったラベルのテキストのみ抽出
for label in model.classes_:
  label = int(label)
  label_texts.append(gestures[label])

#列名をテキストにする
df_y_prob.columns = label_texts

#プロット
df_y_prob.plot()